In [92]:
from langgraph.graph import START,END,StateGraph
from pydantic import BaseModel,Field
from dotenv import load_dotenv
from langchain_core.messages import SystemMessage,HumanMessage 
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import Optional

from typing import Literal
load_dotenv()
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [93]:
generator_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
evaluator_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
optimiser_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")



In [94]:

class TweetState(BaseModel):
    topic: str = Field(..., description="topic given by user")  # required
    tweet: Optional[str] = Field(default=None, description="generated content on topic")
    feedback: Optional[str] = None
    evaluation: Optional[Literal["approved", "not_approved"]] = None
    itrn: int = 0                     # current iteration counter
    max_itrn: int = 5                 # max iterations (prevents 'Field required')


In [95]:
class EvaluateDTO(BaseModel):
    evaluation:Literal["approved","not_approved"]= Field(description="this is the evaluation where you have to evaluate and tell is this okay to go with this or not ")
    feedback:str=Field(description="give proper feedback in this ")
   
    
structured_evaluator_llm =evaluator_llm.with_structured_output(EvaluateDTO)
    

In [96]:
def generate_tweet(state:TweetState):
    prompt= f'''you are an twitter content generator act as profession content creator and generate tweet on topic {state.topic}   '''
    
    
    messages = [
        SystemMessage(content="You are a funny and clever Twitter/X influencer."),
        HumanMessage(content=f"""
        Write a short, original, and hilarious tweet on the topic: "{state.topic}".

        Rules:
        - Do NOT use question-answer format.
        - Max 280 characters.
        - Use observational humor, irony, sarcasm, or cultural references.
        - Think in meme logic, punchlines, or relatable takes.
        - Use simple, day to day english""")
        ]
    return {"tweet":generator_llm.invoke(messages).content}


In [97]:
# evaluator llm 
def evaluate_tweet(state:TweetState):
    messages = [
    SystemMessage(content="You are a ruthless, no-laugh-given Twitter critic. You evaluate tweets based on humor, originality, virality, and tweet format."),
    HumanMessage(content=f"""
    Evaluate the following tweet:

    Tweet: "{state.tweet}"

    Use the criteria below to evaluate the tweet:

    1. Originality – Is this fresh, or have you seen it a hundred times before?  
    2. Humor – Did it genuinely make you smile, laugh, or chuckle?  
    3. Punchiness – Is it short, sharp, and scroll-stopping?  
    4. Virality Potential – Would people retweet or share it?  
    5. Format – Is it a well-formed tweet (not a setup-punchline joke, not a Q&A joke, and under 280 characters)?

    Auto-reject if:
    - It's written in question-answer format (e.g., "Why did..." or "What happens when...")
    - It exceeds 280 characters
    - It reads like a traditional setup-punchline joke
    - Dont end with generic, throwaway, or deflating lines that weaken the humor (e.g., “Masterpieces of the auntie-uncle universe” or vague summaries)

    ### Respond ONLY in structured format:
    - evaluation: "approved" or "needs_improvement"  
    - feedback: One paragraph explaining the strengths and weaknesses 
    """)
    ]

    res:EvaluateDTO= structured_evaluator_llm.invoke(messages)
    
    return {"evaluation":res.evaluation, "feedback":res.feedback}


In [98]:
def optimise_tweet(state:TweetState):
    messages = [
    SystemMessage(content="You punch up tweets for virality and humor based on given feedback."),
    HumanMessage(content=f"""
    Improve the tweet based on this feedback:
    "{state['feedback']}"

    Topic: "{state.topic}"
    Original Tweet:
    {state.tweet}

    Re-write it as a short, viral-worthy tweet. Avoid Q&A style and stay under 280 characters.
    """)
        ]

    response = optimiser_llm.invoke(messages).content
    iteration = state.itrn + 1
    
    return {"tweet":response,"itrn":iteration}


In [ ]:
def check_next(state:TweetState):
    if state.evaluation=="approved":
        return "approved"
    else :
        return "not_approved"

graph= StateGraph(TweetState)
graph.add_node("generate_tweet",generate_tweet)
graph.add_node("evaluate_tweet",evaluate_tweet)
graph.add_node("optimise_tweet",optimise_tweet)

graph.add_edge(START,"generate_tweet")
graph.add_edge("generate_tweet","evaluate_tweet")

graph.add_conditional_edges("evaluate_tweet",check_next,{"approved":END,"not_approved":"optimise_tweet"})
graph.add_edge("optimise_tweet","evaluate_tweet")

workflow= graph.compile()

res=workflow.invoke({"topic":"indian railway write it like so that it can optimise in second prompt","itrn":1})

print(res)


